# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [45]:
## install finrl library
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [46]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from rl_traders.finrl.finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from rl_traders.finrl.finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from rl_traders.finrl.finrl import config_tickers
from rl_traders.finrl.finrl.config import INDICATORS

import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [47]:
aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

[*********************100%***********************]  1 of 1 completed


In [48]:
aapl_df_yf.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.716072,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.009109,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.582901,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.241562,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.403664,132079200


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [49]:
aapl_df_finrl = YahooDownloader(start_date = '2021-08-17',
                                end_date = '2022-10-05',
                                ticker_list = ['aapl']).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (286, 8)


In [50]:
aapl_df_finrl.head()

,date,open,high,low,close,volume,tic,day
0,2021-08-17,150.229996,151.679993,149.089996,147.392029,92229700,aapl,1
1,2021-08-18,149.800003,150.720001,146.149994,143.633377,86326000,aapl,2
2,2021-08-19,145.029999,148.000000,144.500000,143.967056,86960300,aapl,3
3,2021-08-20,147.440002,148.500000,146.779999,145.429291,60549600,aapl,4
4,2021-08-23,148.309998,150.190002,147.889999,146.920975,60131800,aapl,0


## Data for the chosen tickers

In [51]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [52]:
# TRAIN_START_DATE = '2021-08-17'
TRAIN_START_DATE = '2019-10-05'
TRAIN_END_DATE = '2022-10-05'
TRADE_START_DATE = '2022-10-06'
TRADE_END_DATE = '2023-04-10'

In [53]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = ["MSFT"]).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (882, 8)


In [54]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2019-10-07,137.139999,138.179993,137.020004,130.708496,15303700,MSFT,0
1,2019-10-08,137.080002,137.759995,135.619995,129.326294,25550500,MSFT,1
2,2019-10-09,137.460007,138.699997,136.970001,131.776123,19749900,MSFT,2
3,2019-10-10,138.490005,139.669998,138.250000,132.595963,17654600,MSFT,3
4,2019-10-11,140.119995,141.029999,139.500000,133.148819,25446000,MSFT,4


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [55]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (881, 8)
Successfully added vix


Successfully added turbulence index


In [56]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [57]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2019-10-07,MSFT,137.139999,138.179993,137.020004,130.708496,15303700.0,0.0,0.000000,131.972124,128.062666,0.000000,-66.666667,100.000000,130.708496,130.708496,17.860001,0.0
1,2019-10-08,MSFT,137.080002,137.759995,135.619995,129.326294,25550500.0,1.0,-0.031011,131.972124,128.062666,0.000000,-66.666667,100.000000,130.017395,130.017395,20.280001,0.0
2,2019-10-09,MSFT,137.460007,138.699997,136.970001,131.776123,19749900.0,2.0,0.036455,133.060190,128.147086,64.708304,79.040904,16.432655,130.603638,130.603638,18.639999,0.0
3,2019-10-10,MSFT,138.490005,139.669998,138.250000,132.595963,17654600.0,3.0,0.095846,133.928812,128.274625,71.167259,110.455964,20.301135,131.101719,131.101719,17.570000,0.0
4,2019-10-11,MSFT,140.119995,141.029999,139.500000,133.148819,25446000.0,4.0,0.148100,134.568399,128.453879,74.431630,115.903140,45.887919,131.511139,131.511139,15.580000,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [58]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

755
125


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [59]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')